<a href="https://colab.research.google.com/github/HKubinhetz/DS-Seaborn-and-Linear-Regression/blob/main/Seaborn_and_Linear_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

Do higher film budgets lead to more box office revenue? Let's find out if there's a relationship using the movie budgets and financial performance data that I've scraped from [the-numbers.com](https://www.the-numbers.com/movie/budgets) on **May 1st, 2018**. 

<img src=https://i.imgur.com/kq7hrEh.png>

# Import Statements

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# Notebook Presentation

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# Read the Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data = pd.read_csv('/content/drive/MyDrive/PythonDS/07 - Seaborn and Linear Regression/cost_revenue_dirty.csv')

MessageError: ignored

# Explore and Clean the Data

**Challenge**: Answer these questions about the dataset:
1. How many rows and columns does the dataset contain?
2. Are there any NaN values present?
3. Are there any duplicate rows?
4. What are the data types of the columns?

In [ ]:
# How many rows and columns does the dataset contain?
data.shape

In [ ]:
# Are there any NaN values present?
data.isna().any()

In [ ]:
# Are there any duplicate rows?
data.duplicated().any()

In [ ]:
# What are the data types of the columns?
data.info()

### Data Type Conversions

**Challenge**: Convert the `USD_Production_Budget`, `USD_Worldwide_Gross`, and `USD_Domestic_Gross` columns to a numeric format by removing `$` signs and `,`. 
<br>
<br>
Note that *domestic* in this context refers to the United States.

In [ ]:
data.sample(5)

In [ ]:
clean_df = data
clean_df["USD_Production_Budget"] = pd.to_numeric(clean_df["USD_Production_Budget"].str.replace('$', "").str.replace(',', ""))
clean_df["USD_Worldwide_Gross"] = pd.to_numeric(clean_df["USD_Worldwide_Gross"].str.replace('$', "").replace(',', "").str.replace(',', ""))
clean_df["USD_Domestic_Gross"] = pd.to_numeric(clean_df["USD_Domestic_Gross"].str.replace('$', "").replace(',', "").str.replace(',', ""))
clean_df.sample(5)

In [ ]:
clean_df.info()

**Challenge**: Convert the `Release_Date` column to a Pandas Datetime type. 

### Descriptive Statistics

In [ ]:
clean_df["Release_Date"] = pd.to_datetime(clean_df["Release_Date"])
clean_df.sample(5)

In [ ]:
clean_df.info()

**Challenge**: 

1. What is the average production budget of the films in the data set?
2. What is the average worldwide gross revenue of films?
3. What were the minimums for worldwide and domestic revenue?
4. Are the bottom 25% of films actually profitable or do they lose money?
5. What are the highest production budget and highest worldwide gross revenue of any film?
6. How much revenue did the lowest and highest budget films make?

In [ ]:
# Most of the answers can be found through the 'describe' method. Now it is up to us to interpret this data.
clean_df.describe()

In [ ]:
# What is the average production budget of the films in the data set?
print(f"The average production budget of the films in this data set is {'{0:.2f}'.format(clean_df['USD_Production_Budget'].mean())} USD.")

In [ ]:
# What is the average worldwide gross revenue of films?
print(f"The average gross revenue in this data set is {'{0:.2f}'.format(clean_df['USD_Worldwide_Gross'].mean())} USD.")

In [ ]:
# What were the minimums for worldwide and domestic revenue?
clean_df['USD_Worldwide_Gross'].min()
clean_df['USD_Domestic_Gross'].min()

In [ ]:
# Are the bottom 25% of films actually profitable or do they lose money?
print(clean_df.quantile(0.25))
print("The data above shows that on average, the bottom 25% spends 5MM and profits 5,195MM.")

In [ ]:
# What are the highest production budget and highest worldwide gross revenue of any film?
# Highest Production Budget
clean_df.sort_values(by='USD_Production_Budget', ascending=False).head(1)

In [ ]:
# Highest Worldwide Gross Revenue
clean_df.sort_values(by='USD_Worldwide_Gross', ascending=False).head(1)

In [ ]:
# How much revenue did the lowest and highest budget films make?
clean_df.sort_values(by='USD_Production_Budget', ascending=True).head(1)

The highest gross Revenue and Budget both belong to Avatar. The lowest budget is from a movie called My Date With Drew. The movie profited around 165 times its budget.

# Investigating the Zero Revenue Films

**Challenge** How many films grossed $0 domestically (i.e., in the United States)? What were the highest budget films that grossed nothing?

In [ ]:
zero_revenue_movies = clean_df[clean_df['USD_Domestic_Gross']==0].sort_values(by='USD_Production_Budget', ascending=False)
zero_revenue_movies_count = zero_revenue_movies['USD_Domestic_Gross'].count()

print(f"There were {zero_revenue_movies_count} movies that grossed $0 domestically.")

In [ ]:
zero_revenue_movies.head(5)

**Challenge**: How many films grossed $0 worldwide? What are the highest budget films that had no revenue internationally?

In [ ]:
zero_revenue_movies_internationally = clean_df[clean_df['USD_Worldwide_Gross']==0].sort_values(by='USD_Production_Budget', ascending=False)
zero_revenue_movies_count_internationally = zero_revenue_movies_internationally["Movie_Title"].count()

print(f"There were {zero_revenue_movies_count_internationally} movies that grossed $0 internationally.")

In [ ]:
zero_revenue_movies_internationally.head(5)

### Filtering on Multiple Conditions

In [ ]:
international_releases = clean_df.loc[(clean_df.USD_Domestic_Gross == 0) & 
                                  (clean_df.USD_Worldwide_Gross != 0)]
international_releases

**Challenge**: Use the [`.query()` function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html) to accomplish the same thing. Create a subset for international releases that had some worldwide gross revenue, but made zero revenue in the United States. 

Hint: This time you'll have to use the `and` keyword.

In [ ]:
clean_df.query('USD_Worldwide_Gross!=0 and USD_Domestic_Gross==0')

### Unreleased Films

**Challenge**:
* Identify which films were not released yet as of the time of data collection (May 1st, 2018).
* How many films are included in the dataset that have not yet had a chance to be screened in the box office? 
* Create another DataFrame called data_clean that does not include these films. 

In [ ]:
# Date of Data Collection
scrape_date = pd.Timestamp('2018-5-1')

In [ ]:
future_releases = clean_df[clean_df['Release_Date']>=scrape_date]

print(f"There were {len(future_releases)} non-released movies by the time this data was scraped.")

In [ ]:
# Dropping unreleased movies:
data_clean = data.drop(future_releases.index)

### Films that Lost Money

**Challenge**: 
What is the percentage of films where the production costs exceeded the worldwide gross revenue? 

In [ ]:
losing_films = data_clean.query('USD_Production_Budget > USD_Worldwide_Gross')
losing_films

In [ ]:
losing_movies_ratio = len(losing_films)/len(data_clean)*100
losing_movies_percentage = '{:.2f}'.format(losing_movies_ratio)

print(f"The study shows that {losing_movies_percentage}% of all movies actually lose money.")

# Seaborn for Data Viz: Bubble Charts

In [ ]:
plt.figure(figsize=(8,4), dpi=200)
scatterplot = sns.scatterplot(data=data_clean,
                x='USD_Production_Budget', 
                y='USD_Worldwide_Gross',
                hue='USD_Worldwide_Gross', # colour
                size='USD_Worldwide_Gross') # dot size

scatterplot.set(ylim=(0, 3000000000),
       xlim=(0, 450000000),
       ylabel='Revenue in $ billions',
       xlabel='Budget in $100 millions')
 
plt.show()

### Plotting Movie Releases over Time

**Challenge**: Try to create the following Bubble Chart:

<img src=https://i.imgur.com/8fUn9T6.png>



# Converting Years to Decades Trick

**Challenge**: Create a column in `data_clean` that has the decade of the release. 

<img src=https://i.imgur.com/0VEfagw.png width=650> 

Here's how: 
1. Create a [`DatetimeIndex` object](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DatetimeIndex.html) from the Release_Date column. 
2. Grab all the years from the `DatetimeIndex` object using the `.year` property.
<img src=https://i.imgur.com/5m06Ach.png width=650>
3. Use floor division `//` to convert the year data to the decades of the films.
4. Add the decades as a `Decade` column to the `data_clean` DataFrame.

### Separate the "old" (before 1969) and "New" (1970s onwards) Films

**Challenge**: Create two new DataFrames: `old_films` and `new_films`
* `old_films` should include all the films before 1969 (up to and including 1969)
* `new_films` should include all the films from 1970 onwards
* How many films were released prior to 1970?
* What was the most expensive film made prior to 1970?

# Seaborn Regression Plots

**Challenge**: Use Seaborn's `.regplot()` to show the scatter plot and linear regression line against the `new_films`. 
<br>
<br>
Style the chart

* Put the chart on a `'darkgrid'`.
* Set limits on the axes so that they don't show negative values.
* Label the axes on the plot "Revenue in \$ billions" and "Budget in \$ millions".
* Provide HEX colour codes for the plot and the regression line. Make the dots dark blue (#2f4b7c) and the line orange (#ff7c43).

Interpret the chart

* Do our data points for the new films align better or worse with the linear regression than for our older films?
* Roughly how much would a film with a budget of $150 million make according to the regression line?

# Run Your Own Regression with scikit-learn

$$ REV \hat ENUE = \theta _0 + \theta _1 BUDGET$$

**Challenge**: Run a linear regression for the `old_films`. Calculate the intercept, slope and r-squared. How much of the variance in movie revenue does the linear model explain in this case?

# Use Your Model to Make a Prediction

We just estimated the slope and intercept! Remember that our Linear Model has the following form:

$$ REV \hat ENUE = \theta _0 + \theta _1 BUDGET$$

**Challenge**:  How much global revenue does our model estimate for a film with a budget of $350 million? 